The dataset is [here](https://github.com/google-research/google-research/tree/master/goemotions)

In [ ]:
from google.colab import auth

auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

Streaming output truncated to the last 5000 lines.
google-cloud-sdk/lib/surface/scc/findings/update_marks.yaml
google-cloud-sdk/lib/surface/scc/muteconfigs/__init__.py
google-cloud-sdk/lib/surface/scc/muteconfigs/create.yaml
google-cloud-sdk/lib/surface/scc/muteconfigs/delete.yaml
google-cloud-sdk/lib/surface/scc/muteconfigs/get.yaml
google-cloud-sdk/lib/surface/scc/muteconfigs/list.yaml
google-cloud-sdk/lib/surface/scc/muteconfigs/update.yaml
google-cloud-sdk/lib/surface/scc/notifications/__init__.py
google-cloud-sdk/lib/surface/scc/notifications/create.yaml
google-cloud-sdk/lib/surface/scc/notifications/delete.yaml
google-cloud-sdk/lib/surface/scc/notifications/describe.yaml
google-cloud-sdk/lib/surface/scc/notifications/list.yaml
google-cloud-sdk/lib/surface/scc/notifications/update.yaml
google-cloud-sdk/lib/surface/scc/operations/__init__.py
google-cloud-sdk/lib/surface/scc/operations/describe.yaml
google-cloud-sdk/lib/surface/scc/settings/__init__.py
google-cloud-sdk/lib/surface/s

In [ ]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
auth:
  service_account_use_self_signed_jwt: None
component_manager:
  disable_update_check: 'True'
core:
  account: pallav.gupta2021@vitstudent.ac.in

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  2

Enter configuration name. Names start with a lower case letter and contain only 
lower case letters a-z, digits 0-9, and hyphens '-':  emotion-analysis
Your current configuration has been set to: [emotion-analysis]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for this 
configuration:
 [1] pa

In [ ]:
!gsutil cp -r gs://gresearch/goemotions/data/full_dataset/ .

Copying gs://gresearch/goemotions/data/full_dataset/goemotions_1.csv...
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_2.csv...
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_3.csv...
- [3 files][ 40.8 MiB/ 40.8 MiB]                                                
Operation completed over 3 objects/40.8 MiB.                                     


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report

Will use goemotion1, goemotion2 to train and goemotion3 to test

In [ ]:
df_train = pd.read_csv("/content/full_dataset/goemotions_1.csv")
df_train.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
df_train.columns

Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   text                  70000 non-null  object 
 1   id                    70000 non-null  object 
 2   author                70000 non-null  object 
 3   subreddit             70000 non-null  object 
 4   link_id               70000 non-null  object 
 5   parent_id             70000 non-null  object 
 6   created_utc           70000 non-null  float64
 7   rater_id              70000 non-null  int64  
 8   example_very_unclear  70000 non-null  bool   
 9   admiration            70000 non-null  int64  
 10  amusement             70000 non-null  int64  
 11  anger                 70000 non-null  int64  
 12  annoyance             70000 non-null  int64  
 13  approval              70000 non-null  int64  
 14  caring                70000 non-null  int64  
 15  confusion          

In [ ]:
df = df_train.drop(['id','author','subreddit','link_id','parent_id','created_utc','rater_id'],axis = 1)

In [ ]:
df['example_very_unclear'].unique()

array([False,  True])

In [ ]:
mat = {False:0,True:1}
df['example_very_unclear'] = df['example_very_unclear'].map(mat)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   text                  70000 non-null  object
 1   example_very_unclear  70000 non-null  int64 
 2   admiration            70000 non-null  int64 
 3   amusement             70000 non-null  int64 
 4   anger                 70000 non-null  int64 
 5   annoyance             70000 non-null  int64 
 6   approval              70000 non-null  int64 
 7   caring                70000 non-null  int64 
 8   confusion             70000 non-null  int64 
 9   curiosity             70000 non-null  int64 
 10  desire                70000 non-null  int64 
 11  disappointment        70000 non-null  int64 
 12  disapproval           70000 non-null  int64 
 13  disgust               70000 non-null  int64 
 14  embarrassment         70000 non-null  int64 
 15  excitement            70000 non-null

In [ ]:
df[df['text']== 'hi']['text']

Series([], Name: text, dtype: object)

In [ ]:
for i in df['text']:
  for j in df.columns[2:]:
      print(df[df['text'] == i][j] == 1)
      break

Streaming output truncated to the last 5000 lines.
Name: admiration, dtype: bool
28886    False
68351    False
Name: admiration, dtype: bool
17404    False
45009    False
68352    False
Name: admiration, dtype: bool
48092    False
68353    False
Name: admiration, dtype: bool
68354    False
Name: admiration, dtype: bool
34188     True
68355    False
Name: admiration, dtype: bool
13830    False
68356    False
Name: admiration, dtype: bool
5961     False
24782    False
68357    False
Name: admiration, dtype: bool
49746    False
68358    False
Name: admiration, dtype: bool
48007    False
68359    False
Name: admiration, dtype: bool
42210    False
68360    False
Name: admiration, dtype: bool
50547    False
53408    False
68361    False
Name: admiration, dtype: bool
57317    False
58166    False
62680    False
67536    False
68362    False
Name: admiration, dtype: bool
30358    False
68363    False
Name: admiration, dtype: bool
30198    False
49410    False
50348    False
51236    False
5487

In [ ]:
for i in df['text']:
  for j in range(2,len(df.columns)):
    if (df[df['text'] == i][df.columns[j]] == 1):
      df['emotion'] = j
      break
    else:
      df['emotion'] = 0

ValueError: ignored